In [1]:
from dolfinx import geometry, fem, mesh, plot, io
from mpi4py import MPI
from petsc4py import PETSc
from petsc4py.PETSc import ScalarType, ComplexType
from ufl import (TrialFunction, Measure, TestFunction, dx, ds, grad, div, inner, lhs, rhs)

import timeit
import numpy as np
import pandas as pd

from utils.dolfinx import BoundaryCondition, generate_boundary_measure, project
from utils.plotting import Mpl1DAnimator

from IPython.display import Video
from pathlib import Path

In [2]:
save_dir = "./results/1d-homogeneous-with-dissipation"
Path(save_dir).mkdir(parents=True, exist_ok=True)

In [3]:
def generate_boundaries(points):
    return [(1, lambda x: np.isclose(x[0], points[0])), 
            (2, lambda x: np.isclose(x[0], points[1]))]

In [4]:
def problem_setup(N: int,
                  points: list[float, float],
                  T: float,
                  dt: float,
                  fluid: str):
    """
    Performs problem configuration w.r.t. given parameters
    """
    
    # Mesh and function space definition
    domain = mesh.create_interval(MPI.COMM_WORLD, N, points)
    V = fem.FunctionSpace(domain, ("CG", 1))

    u = TrialFunction(V)
    v = TestFunction(V)
    
    # Definition of density and speed functions
    ro, c, eta = fluids.loc[fluid, ['Density', 'Speed of sound', 'Viscosity']]
        
    # Construction of bilinear form and linear functional
    GAMMA, BETA = 0.5, 0.5
    
    mm = ScalarType(ro) * inner(u, v) * dx
    aa = ScalarType(ro * c**2) * inner(grad(u), grad(v)) * dx
    cc = ScalarType(4./3 * eta) * inner(grad(u), grad(v)) * dx
    
    F = mm + (dt * GAMMA * cc) + (0.5 * dt * dt * BETA * aa)
        
    boundaries = generate_boundaries(points)
    measure = generate_boundary_measure(boundaries, domain)
    
    u_D = lambda x: x[0] * 0.0
    u_N1 = fem.Constant(domain, ScalarType(-1.0))
    u_N2 = fem.Constant(domain, ScalarType(0.0))
    
    bcs = [BoundaryCondition("Dirichlet", 1, u_D, V, u, v, measure).bc]
    nbcs = [BoundaryCondition("Neumann", 2, u_N1, V, u, v, measure).bc,
            BoundaryCondition("Neumann", 2, u_N2, V, u, v, measure).bc] 
    
    return {
        'Params': (N, points, fluid, ro, c, T, dt, GAMMA, BETA),
        'FunctionSpace': (domain, V, u, v),
        'Forms': (mm, aa, cc), 
        'Problem': (F, bcs, nbcs)
    }

In [5]:
def solve_problem(config):
    N, points, fluid, ro, c, T, dt, GAMMA, BETA = config['Params']
    domain, V, u, v = config['FunctionSpace']
    mm, aa, cc = config['Forms']
    F, bcs, nbcs = config['Problem']
    
    # Create initial condition
    initial_condition = lambda x: x[0] * 0.0
    
    uj = fem.Function(V)
    uj_d = fem.Function(V)
    uj_dd = fem.Function(V)
    uj_d_d_tg = fem.Function(V)
    
    uj.interpolate(initial_condition)
    uj_d.interpolate(initial_condition)
    uj_dd.interpolate(initial_condition)
    uj_d_d_tg.interpolate(initial_condition)
    
    p = project(ro * c**2 * uj.dx(0), domain, ('CG', 1))
    p.x.array[:] = ro * c**2 * uj_d.x.array
    
    # Construct the left and right hand side of the problem
    bilinear_form = fem.form(lhs(F))
    
    A = fem.petsc.assemble_matrix(bilinear_form, bcs=bcs)
    A.assemble()
    
    solver = PETSc.KSP().create(domain.comm)
    solver.setOperators(A)
    solver.setType(PETSc.KSP.Type.PREONLY)
    solver.getPC().setType(PETSc.PC.Type.LU)
    
    # Solve problem at each time step,
    num_steps = int(T / dt)
    t = 0.0
    
    time_data = {
        't'   : [0.0,],
        'uj'  : [np.copy(uj.x.array[:]),],
        'uj_d': [np.copy(uj_d.x.array[:]),],
        'p'   : [np.copy(p.x.array[:]),]
    }
    
    uj_d_d_tg.x.array[:] = dt * GAMMA * uj_d.x.array
        
    L1 = nbcs[0] + (cc * uj_d) + (aa * uj) + (aa * uj_d_d_tg)
    L2 = nbcs[1] + (cc * uj_d) + (aa * uj) + (aa * uj_d_d_tg)
    linear_form_1 = fem.form(rhs(L1))
    linear_form_2 = fem.form(rhs(L2))
    
    b1 = fem.petsc.create_vector(linear_form_1)
    b2 = fem.petsc.create_vector(linear_form_2)
        
    for i in range(num_steps):
        t += dt
        
        # idx = 0
        idx = 0 if t < 1.0e-6 else 1
        
        uj_d_d_tg.x.array[:] = dt * GAMMA * uj_d.x.array
        
        b = b1 if t < 1.5e-6 else b2
        linear_form = linear_form_1 if t < 1.5e-6 else linear_form_2

        with b.localForm() as loc_b:
            loc_b.set(0)
        fem.petsc.assemble_vector(b, linear_form)

        # Apply Dirichlet boundary condition to the vector
        fem.petsc.apply_lifting(b, [bilinear_form], [bcs])
        b.ghostUpdate(addv=PETSc.InsertMode.ADD, mode=PETSc.ScatterMode.REVERSE)
        fem.petsc.set_bc(b, bcs)
        
        # Solve linear problem
        solver.solve(b, uj_dd.vector)
        uj_dd.x.scatter_forward()
        
        # Update solution
        uj.x.array[:] = uj.x.array + dt * uj_d.x.array + 0.5 * dt**2 * uj_dd.x.array
        uj_d.x.array[:] = uj_d.x.array + dt * uj_dd.x.array
        
        if i % 10 == 0:
            p = project(ro * c**2 * uj.dx(0), domain, ('CG', 1))
            
            time_data['t'].append(t)
            time_data['uj'].append(np.copy(uj.x.array))
            time_data['uj_d'].append(np.copy(uj_d.x.array))
            time_data['p'].append(np.copy(p.x.array))

    return uj, uj_d, p, time_data

#### Available fluids

In [6]:
fluids = pd.read_csv('../data/physical_properties.csv', sep=';', index_col='Fluid')
fluids

,Density,Speed of sound,Viscosity,Heat,Thermal conductivity,Thermal expansion
Fluid,,,,,,
Water,1000,1500,0.000894,4182.0,0.598,0.00015
Fuel oil,890,1360,2.022000,2090.0,0.150,0.00070
Oil,760,1470,0.000500,NaN,NaN,NaN
Glycerin,1260,1905,1.500000,NaN,NaN,NaN


In [7]:
config = problem_setup(N=500, 
                       points=[0.0, 0.02],
                       T=5.0e-5,
                       dt=5.0e-8,
                       fluid='Fuel oil')

In [8]:
uj, uj_d, p, time_data = solve_problem(config)
N, points, *_ = config['Params']

In [9]:
mpa = Mpl1DAnimator(layout=[['1'], ['2'], ['3']], time=time_data['t'])
mpa.update_figure(figsize=(16, 10), fontsize=20)
mpa.add_fun(fun=uj, time_data=time_data['uj'], type='real', 
            points=points, N=N, axes_id='1', color='r', title='Displacement')
mpa.add_fun(fun=uj_d, time_data=time_data['uj_d'], type='real', 
            points=points, N=N, axes_id='2', color='g', title='Velocity')
mpa.add_fun(fun=p, time_data=time_data['p'], type='real', 
            points=points, N=N, axes_id='3',  color='b', title='Pressure')
mpa.write(path=save_dir, filename='animation.mp4', fps=10, interval=1000)

In [10]:
Video(f'{save_dir}/animation.mp4', width=800)